<a href="https://colab.research.google.com/github/AndreRFlenik/Analise_De_Concorrencia_Mercado_Livre/blob/main/Analise_de_Concorr%C3%AAncia_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de Bibliotecas


In [101]:
# Importar as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import plotly.express as px


from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler


import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.tree import DecisionTreeRegressor

#Função de Busca no Mercado Livre - WebScraping

In [102]:
def search_mercadolivre(product_name):
    url = f"https://lista.mercadolivre.com.br/{product_name}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.find_all('li', {'class': 'ui-search-layout__item'})

    datasearch = []

    for product in products:
        name_element = product.find('h2', {'class': 'ui-search-item__title'})
        name = name_element.text.strip() if name_element else 'Nome não disponível'

        price_element = product.find('span', {'class': 'andes-money-amount'})
        if price_element:
            price_text = price_element.text.strip()
            price_text = price_text.replace('R$', '').replace('.', '').replace(',', '.').strip()
            price = float(price_text)
        else:
            price = 0.0

        avaliation_element = product.find('span', {'class': 'ui-search-reviews__rating-number'})
        avaliation = avaliation_element.text.strip() if avaliation_element else '0'
        avaliation = float(avaliation)

        opinion_element = product.find('span', {'class': 'ui-search-reviews__amount'})
        if opinion_element:
            opinion_text = opinion_element.text.strip()
            opinion_match = re.search(r'\((\d+)\)', opinion_text)
            opinion = int(opinion_match.group(1)) if opinion_match else 0
        else:
            opinion = 0
        opinion = float(opinion)

        link_element = product.find('a', {'class': 'ui-search-link'})
        link = link_element['href'] if link_element else 'Link não disponível'

        # Nova parte para extrair o número de vendas
        sales = 0
        if link != 'Link não disponível':

            # Requisição para a página do produto
            product_response = requests.get(link)
            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            # Encontrar o elemento que contém as vendas
            sales_element = product_soup.find('span', {'class': 'ui-pdp-subtitle'})
            if sales_element:
                # Extrair o número de vendas usando expressões regulares
                sales_match = re.search(r'\+(\d+)\s+vendidos', sales_element.text)
                sales = int(sales_match.group(1)) if sales_match else 0
                sales= float(sales)

                 # Verificar categoria Full
            full_category = 0
            full_icon = product_soup.find('svg', {'class': 'ui-pdp-icon ui-pdp-icon--full ui-pdp-color--GREEN'})
            if full_icon:
                full_category = 1

        datasearch.append([name, price, avaliation, opinion, sales, full_category])

    return datasearch

#Busca dos Produtos Concorrentes

##Inserção de dados do produto do usuário

In [103]:
# Insira os dados do Produto (Nome, Preço Estimado, se é Full)
produto_nome = 'Notebook'
produto_preco = [2000]
produto_full =  [1]

#OBS.:Quanto mais detalhado o nome do produto maior será a precisão da busca.

# Exemplo: Kit 10 Peças Rolamento 6002 Zz Medida 15x32x9

## Dataframe com todos os produtos Encontrados

In [104]:
# Criação do DataFrame com os produtos concorrentes
df_pesquisa = pd.DataFrame(columns=['produto_nome','produto_preco', 'produto_nota','produto_votantes','produto_vendas', 'produto_full'])

# Chamar a função para preencher o dataframe
data = search_mercadolivre(produto_nome)
df_pesquisa = pd.concat([df_pesquisa, pd.DataFrame(data, columns=['produto_nome','produto_preco', 'produto_nota','produto_votantes','produto_vendas', 'produto_full'])], ignore_index=True)


df_pesquisa

,produto_nome,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full
0,Notebook Samsung Galaxy Book2 Intel Core I5 - ...,2649.0,4.8,3846.0,0.0,1
1,Notebook Lenovo Ideapad 3 R7-5700u 8gb 512gb S...,2819.0,4.8,1564.0,1000.0,0
2,Notebook Positivo Vision C15 Lumina Bar Intel ...,1480.0,4.6,1408.0,0.0,0
3,"Notebook Acer Aspire 3 Intel Core I3 - N305, 8...",2345.0,4.8,3737.0,0.0,1
4,Notebook Gamer Anv15-51-582r Ci5 16gb 512ssd 1...,4800.0,4.8,118.0,500.0,1
5,Notebook Lenovo Ideapad 1i Intel Core I5 - 123...,2829.0,4.8,1383.0,0.0,1
6,Notebook Dell Inspiron I15-i120k-a30pf I5 16gb...,3689.0,4.8,1156.0,0.0,1
7,Notebook Dell Inspiron I15-i120k-a10p 15.6 I3 ...,2749.0,4.8,331.0,1000.0,1
8,"Notebook Acer A515-57-5429 Ci5 16gb 512ssd 15,...",2919.0,4.9,78.0,500.0,1
9,Notebook Dell Latitude 3470 Core I5 + 16 Gb Ra...,1839.0,4.5,177.0,500.0,0


# Clusterização dos Produtos Encontrados


In [105]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px

# Remover a coluna 'produto_nome' do DataFrame para permenecer apenas as variaveis numéricas
df_cluster = df_pesquisa.drop(columns=['produto_nome'])

# Escalar as variáveis
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Instanciar o modelo KMeans com o número desejado de clusters
num_clusters = 3  # Três grupos selecionados
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)

# Ajustar o modelo aos dados escalados
kmeans_model.fit(df_scaled)

# Adicionar rótulos de cluster aos dados
df_cluster['Cluster'] = kmeans_model.labels_

# Adicionar a coluna 'Cluster' ao DataFrame df_pesquisa
df_pesquisa = df_pesquisa.merge(df_cluster[['Cluster']], left_index=True, right_index=True)

# Plotar o gráfico de dispersão com cores diferentes para cada cluster
fig = px.scatter(df_cluster, x='produto_preco', y='produto_nota', color='Cluster', title='Clusters de Produtos')
fig.show()



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



##Verificar o Cluster em que o produto pertence

In [106]:
# Tratamento das variáveis
produto_preco = produto_preco[0]  # Acessa o primeiro elemento da lista, se for o caso
produto_full = produto_full[0]  # Acessa o primeiro elemento da lista, se for o caso

# Dados do novo produto
novo_produto = {
    'produto_nome': [produto_nome],
    'produto_preco': [produto_preco],
    'produto_full': [produto_full]
}

# Converter para DataFrame
df_novo_produto = pd.DataFrame(novo_produto)

# Selecionar as mesmas variáveis que foram usadas para o treinamento
variaveis = ['produto_preco', 'produto_full']

# Verificar se as variáveis estão presentes no DataFrame df_pesquisa
if not all(var in df_pesquisa.columns for var in variaveis):
    raise ValueError("As variáveis não estão presentes no DataFrame df_pesquisa.")

# Normalizar os dados
scaler = StandardScaler()
df_treinamento = df_pesquisa[variaveis]
df_treinamento_scaled = scaler.fit_transform(df_treinamento)

# Normalizar o novo produto
df_novo_produto[variaveis] = df_novo_produto[variaveis].astype(float)  # Converter para tipo float, se necessário
df_novo_produto_scaled = scaler.transform(df_novo_produto[variaveis])

# Treinar o modelo K-means com os dados existentes
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_treinamento_scaled)

# Prever o cluster do novo produto
cluster_novo_produto = kmeans.predict(df_novo_produto_scaled)

print(f'O novo produto pertence ao cluster: {cluster_novo_produto[0]}')





O novo produto pertence ao cluster: 0


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [107]:

cluster_desejado = cluster_novo_produto[0]

# Apenas os produtos do cluster pesquisado
if 'Cluster' in df_pesquisa.columns:
    df_produtos_cluster = df_pesquisa[df_pesquisa['Cluster'] == cluster_desejado].copy()

    df_produtos_cluster

else:
    print("A coluna 'Cluster' não está presente no DataFrame df_pesquisa.")

df_produtos_cluster

,produto_nome,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full,Cluster
1,Notebook Lenovo Ideapad 3 R7-5700u 8gb 512gb S...,2819.0,4.8,1564.0,1000.0,0,0
4,Notebook Gamer Anv15-51-582r Ci5 16gb 512ssd 1...,4800.0,4.8,118.0,500.0,1,0
7,Notebook Dell Inspiron I15-i120k-a10p 15.6 I3 ...,2749.0,4.8,331.0,1000.0,1,0
8,"Notebook Acer A515-57-5429 Ci5 16gb 512ssd 15,...",2919.0,4.9,78.0,500.0,1,0
9,Notebook Dell Latitude 3470 Core I5 + 16 Gb Ra...,1839.0,4.5,177.0,500.0,0,0
10,Notebook Dell Latitude 3490 - I5 + 16 Gb Ddr4 ...,2160.0,5.0,2.0,5.0,0,0
11,Notebook Gamer Barato Ryzen 3 Pro 8gb Ssd 256g...,1592.0,4.7,3.0,5.0,1,0
12,Notebook Asus Vivobook Go Core I3 N305 4gb 256...,2248.0,4.7,76.0,100.0,1,0
13,Notebook Positivo Vision C14 Lumina Bar Intel ...,1299.0,4.6,296.0,0.0,1,0
14,Notebook Dell Inspiron I15-i120k-a15p I3 8gb 5...,2789.0,4.8,78.0,500.0,1,0


In [108]:
# Função para identificar e remover outliers usando IQR
def remove_outliers_iqr(df):
    df_cleaned = df.copy()
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_cleaned = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]
    return df_cleaned

# Remover outliers de todas as colunas numéricas
df_produtos_cluster= remove_outliers_iqr(df_produtos_cluster)
df_produtos_cluster

,produto_nome,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full,Cluster
4,Notebook Gamer Anv15-51-582r Ci5 16gb 512ssd 1...,4800.0,4.8,118.0,500.0,1,0
7,Notebook Dell Inspiron I15-i120k-a10p 15.6 I3 ...,2749.0,4.8,331.0,1000.0,1,0
8,"Notebook Acer A515-57-5429 Ci5 16gb 512ssd 15,...",2919.0,4.9,78.0,500.0,1,0
11,Notebook Gamer Barato Ryzen 3 Pro 8gb Ssd 256g...,1592.0,4.7,3.0,5.0,1,0
12,Notebook Asus Vivobook Go Core I3 N305 4gb 256...,2248.0,4.7,76.0,100.0,1,0
13,Notebook Positivo Vision C14 Lumina Bar Intel ...,1299.0,4.6,296.0,0.0,1,0
14,Notebook Dell Inspiron I15-i120k-a15p I3 8gb 5...,2789.0,4.8,78.0,500.0,1,0
17,Notebook Acer Aspire 5 Intel Core I5 12450H 8...,2799.0,4.8,611.0,1000.0,1,0
21,Notebook Gamer Dell G15-i1300-a20p I5 8gb 512g...,5608.0,4.8,362.0,1000.0,1,0
23,Notebook Positivo Master N1250 Pentium N5030 4...,1369.0,4.6,161.0,1000.0,1,0


# Análise Descritiva

In [109]:
df_produtos_cluster.describe()

,produto_preco,produto_nota,produto_votantes,produto_vendas,Cluster
count,28.000000,28.000000,28.000000,28.000000,28.0
mean,3161.428571,4.778571,190.142857,546.607143,0.0
std,1249.531036,0.091721,161.142154,406.621249,0.0
min,1299.000000,4.600000,3.000000,0.000000,0.0
25%,2368.750000,4.700000,77.500000,100.000000,0.0
50%,2794.000000,4.800000,128.000000,500.000000,0.0
75%,4004.750000,4.800000,298.250000,1000.000000,0.0
max,5890.000000,4.900000,611.000000,1000.000000,0.0


##Comparação de Preços

In [110]:

# Calcular o indicador com base na média
produto_indicador_media = (float(produto_preco) / float(df_produtos_cluster['produto_preco'].mean())) * 100

# Condicional para imprimir o resultado
if produto_indicador_media > 100:
    print(f"O preço do produto é: {(produto_indicador_media-100):.2f}% maior que a média")
elif produto_indicador_media < 100:
    print(f"O preço do produto é: {produto_indicador_media:.2f}% menor que a média")
else:
    print(f"O preço do produto é igual à média")

# Calcular o indicador com base na mediana
produto_indicador_mediana = (float(produto_preco) / float(df_produtos_cluster['produto_preco'].median())) * 100
# Condicional para imprimir o resultado
if produto_indicador_mediana > 100:
    print(f"O preço do produto é: {(produto_indicador_mediana-100):.2f}% maior que a mediana")
elif produto_indicador_mediana < 100:
    print(f"O preço do produto é: {produto_indicador_mediana:.2f}% menor que a mediana")
else:
    print(f"O preço do produto é igual à mediana")

# Calcular o indicador com base no mínimo
produto_indicador_min = (float(produto_preco) / float(df_produtos_cluster['produto_preco'].min())) * 100
print(f"O preço do produto é: {(produto_indicador_min-100):.2f}% maior que o menor preço")

# Calcular o indicador com base no máximo
produto_indicador_max = (float(df_produtos_cluster['produto_preco'].max()/float(produto_preco))) * 100
print(f"O preço do produto é: {produto_indicador_max:.2f}% menor que o maior preço")

O preço do produto é: 63.26% menor que a média
O preço do produto é: 71.58% menor que a mediana
O preço do produto é: 53.96% maior que o menor preço
O preço do produto é: 294.50% menor que o maior preço


##Localização de Quartil

In [111]:
# Calcular os quartis de produto_preco
quartis_preco = pd.qcut(df_produtos_cluster['produto_preco'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Adicionar quartis ao DataFrame
df_produtos_cluster['Quartil_Preco'] = quartis_preco

# Agrupar por quartil de preço e calcular estatísticas de produto_vendas e produto_nota
estatisticas_quartil = df_produtos_cluster.groupby('Quartil_Preco').agg({
    'produto_vendas': ['mean', 'median', 'max'],
    'produto_nota': ['mean', 'median', 'max']
}).reset_index()


# Valor da variável independente que você quer verificar
valor_independente = produto_preco

# Calcular os quartis
quartis = df_produtos_cluster['produto_preco'].quantile([0.25, 0.5, 0.75])

# Definir os limites dos quartis
q1, q2, q3 = quartis.iloc[0], quartis.iloc[1], quartis.iloc[2]

# Verificar em qual quartil o valor pertence
if valor_independente <= q1:
    quartil = 'Q1'
elif valor_independente <= q2:
    quartil = 'Q2'
elif valor_independente <= q3:
    quartil = 'Q3'
else:
    quartil = 'Q4'

# Resultado
print(f'O Preço do produto pertence ao quartil {quartil}')

estatisticas_quartil



O Preço do produto pertence ao quartil Q1


Quartil_Preco produto_vendas                 produto_nota            
                          mean  median     max         mean median  max
0            Q1     386.428571   100.0  1000.0     4.714286    4.7  4.9
1            Q2     728.571429  1000.0  1000.0     4.785714    4.8  4.9
2            Q3     657.142857   500.0  1000.0     4.785714    4.8  4.9
3            Q4     414.285714   100.0  1000.0     4.828571    4.8  4.9

In [112]:
df_produtos_cluster

,produto_nome,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full,Cluster,Quartil_Preco
4,Notebook Gamer Anv15-51-582r Ci5 16gb 512ssd 1...,4800.0,4.8,118.0,500.0,1,0,Q4
7,Notebook Dell Inspiron I15-i120k-a10p 15.6 I3 ...,2749.0,4.8,331.0,1000.0,1,0,Q2
8,"Notebook Acer A515-57-5429 Ci5 16gb 512ssd 15,...",2919.0,4.9,78.0,500.0,1,0,Q3
11,Notebook Gamer Barato Ryzen 3 Pro 8gb Ssd 256g...,1592.0,4.7,3.0,5.0,1,0,Q1
12,Notebook Asus Vivobook Go Core I3 N305 4gb 256...,2248.0,4.7,76.0,100.0,1,0,Q1
13,Notebook Positivo Vision C14 Lumina Bar Intel ...,1299.0,4.6,296.0,0.0,1,0,Q1
14,Notebook Dell Inspiron I15-i120k-a15p I3 8gb 5...,2789.0,4.8,78.0,500.0,1,0,Q2
17,Notebook Acer Aspire 5 Intel Core I5 12450H 8...,2799.0,4.8,611.0,1000.0,1,0,Q3
21,Notebook Gamer Dell G15-i1300-a20p I5 8gb 512g...,5608.0,4.8,362.0,1000.0,1,0,Q4
23,Notebook Positivo Master N1250 Pentium N5030 4...,1369.0,4.6,161.0,1000.0,1,0,Q1


## Correlação Linear

In [113]:
# Remover valores ausentes
correlation_matrix = df_produtos_cluster[['produto_preco', 'produto_nota', 'produto_votantes', 'produto_vendas', 'produto_full']].dropna()

# Calcular a matriz de correlação
correlations = correlation_matrix.corr()

# Exibir a correlação de 'produto_vendas' com as outras variáveis
for col in correlations.columns:
    if col != 'produto_vendas':
        correlation = correlations.loc['produto_vendas', col]
        print(f'Correlação entre produto_vendas e {col}: {correlation}')

correlations

Correlação entre produto_vendas e produto_preco: 0.038116604050197865
Correlação entre produto_vendas e produto_nota: 0.07692721796265808
Correlação entre produto_vendas e produto_votantes: 0.6929221691511446
Correlação entre produto_vendas e produto_full: -0.004437485250525502


,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full
produto_preco,1.000000,0.478881,0.003249,0.038117,-0.136652
produto_nota,0.478881,1.000000,-0.005298,0.076927,-0.220884
produto_votantes,0.003249,-0.005298,1.000000,0.692922,0.043925
produto_vendas,0.038117,0.076927,0.692922,1.000000,-0.004437
produto_full,-0.136652,-0.220884,0.043925,-0.004437,1.000000


#Análise Preditiva

## Regressão Linear Múltipla

In [114]:


# Selecionar as features e a variável target
X = df_produtos_cluster[['produto_preco', 'produto_nota', 'produto_votantes', 'produto_full']]
y = df_produtos_cluster['produto_vendas']

# Converter para tipos numéricos se necessário
X = X.astype(float)
y = y.astype(float)

# Adicionar uma constante para o termo de intercepto
X = sm.add_constant(X)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de regressão linear multivariável usando statsmodels
model = sm.OLS(y_train, X_train).fit()

# Fazer previsões para o conjunto de teste
X_test = sm.add_constant(X_test)  # Adicionar constante ao conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
mse_reg_multipla = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse_reg_multipla}')

# Prever vendas para um novo produto com características específicas
preco_especifico = produto_preco
nota_especifica = df_produtos_cluster['produto_nota'].mean()
votantes_especifico = df_produtos_cluster['produto_votantes'].mean()
full_especifico = produto_full

# Criar um DataFrame com as características específicas do novo produto
novo_produto = pd.DataFrame({
    'const': [1],  # constante adicionada anteriormente
    'produto_preco': [preco_especifico],
    'produto_nota': [nota_especifica],
    'produto_votantes': [votantes_especifico],
    'produto_full': [full_especifico]
})

# Converter para array NumPy
novo_produto = np.asarray(novo_produto)

# Fazer a previsão para o novo produto
vendas_previstas = model.predict(novo_produto)
print(f'Vendas previstas para o novo produto com preço {preco_especifico}, nota {nota_especifica}, votantes {votantes_especifico} e full {full_especifico}: {vendas_previstas[0]}')
vendas_previstas_reg_multipla = vendas_previstas[0]

# Resumo do modelo
print(model.summary())



Mean Squared Error: 134681.1297304108
Vendas previstas para o novo produto com preço 2000, nota 4.778571428571429, votantes 190.14285714285714 e full 1: 521.6455158355049
                            OLS Regression Results                            
Dep. Variable:         produto_vendas   R-squared:                       0.526
Model:                            OLS   Adj. R-squared:                  0.414
Method:                 Least Squares   F-statistic:                     4.713
Date:                Fri, 05 Jul 2024   Prob (F-statistic):            0.00963
Time:                        12:13:55   Log-Likelihood:                -154.56
No. Observations:                  22   AIC:                             319.1
Df Residuals:                      17   BIC:                             324.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err 

## Árvore de Decisões

In [115]:


# Selecionar as features e a variável target
X = df_produtos_cluster[['produto_preco', 'produto_nota', 'produto_votantes', 'produto_full']]
y = df_produtos_cluster['produto_vendas']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de árvore de decisão
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mse_arvore = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse_arvore}')

produto_nota= df_produtos_cluster['produto_nota'].mean()
produto_votantes= df_produtos_cluster['produto_votantes'].mean()
# Criar um DataFrame com as características específicas do novo produto
novo_produto = pd.DataFrame({
    'produto_nome': [produto_nome],
    'produto_preco': [produto_preco],
    'produto_nota': [produto_nota],  # Incluir todas as features usadas no treinamento
    'produto_votantes': [produto_votantes],
    'produto_full': [produto_full]
})

# Excluir a coluna 'produto_nome' para fazer a previsão
X_novo = novo_produto.drop(['produto_nome'], axis=1)

# Fazer a previsão
vendas_previstas = model.predict(X_novo)
print(f'Vendas previstas para o produto {produto_nome} com preço {produto_preco}: {vendas_previstas[0]}')

vendas_previstas_arvore = vendas_previstas[0]


Mean Squared Error: 328333.3333333333
Vendas previstas para o produto Notebook com preço 2000: 1000.0


## Máquina de Vetores de Suporte

In [116]:


# Selecionar as features e a variável target
X = df_produtos_cluster[['produto_preco', 'produto_nota', 'produto_votantes', 'produto_full']]
y = df_produtos_cluster['produto_vendas']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar e treinar o modelo SVR
model = SVR(kernel='linear')
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mse_svm = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse_svm}')

# Valores médios para características específicas do novo produto

produto_nota_medio = df_produtos_cluster['produto_nota'].mean()
produto_votantes_medio = df_produtos_cluster['produto_votantes'].mean()


# Criar um DataFrame com as características específicas do novo produto
novo_produto = pd.DataFrame({
    'produto_preco': [produto_preco],
    'produto_nota': [produto_nota_medio],  # Usando média para notas e votantes
    'produto_votantes': [produto_votantes_medio],
    'produto_full': [produto_full]
})

# Fazer a previsão para o novo produto
vendas_previstas = model.predict(novo_produto)
print(f'Vendas previstas para o produto com preço {produto_preco}, nota média {produto_nota_medio}, votantes médio {produto_votantes_medio}: {vendas_previstas[0]}')

vendas_previstas_svm = vendas_previstas[0]

Mean Squared Error: 81396.01961859346
Vendas previstas para o produto com preço 2000, nota média 4.778571428571429, votantes médio 190.14285714285714: 580.557281983282


## Indicador Combinado - Previsões e Erros

In [117]:


# Normalização dos EMQs
max_mse = max(mse_reg_multipla, mse_arvore, mse_svm)
nemq_reg_multipla = mse_reg_multipla / max_mse
nemq_arvore = mse_arvore / max_mse
nemq_svm = mse_svm / max_mse

# Cálculo do indicador
indicador = (
    vendas_previstas_reg_multipla * (1 / nemq_reg_multipla) +
    vendas_previstas_arvore * (1 / nemq_arvore) +
    vendas_previstas_svm * (1 / nemq_svm)
) / (
    (1 / nemq_reg_multipla) + (1 / nemq_arvore) + (1 / nemq_svm)
)

print(f"O valor de vendas do indicador combinado é: {indicador:.2f}")

O valor de vendas do indicador combinado é: 617.47
